### Import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

### Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

### Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)
#dropout 기능 추가
def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

### Create model

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        # 2->4 더 깊은 학습
        for i in range(4):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5))# 50% dropout

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    # weight 결정시 glorot_uniform 사용 (xavier)
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(256, weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))# 50% dropout


    model.add(dense(label_dim, weight_init))

    return model

### Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

### Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.7722, train_loss: 2.37157631, train_accuracy: 0.0547, test_Accuracy: 0.0856
Epoch: [ 0] [    1/  468] time: 2.0894, train_loss: 2.35085440, train_accuracy: 0.1719, test_Accuracy: 0.1574
Epoch: [ 0] [    2/  468] time: 2.5452, train_loss: 2.38289547, train_accuracy: 0.2266, test_Accuracy: 0.2244
Epoch: [ 0] [    3/  468] time: 2.9321, train_loss: 2.30179739, train_accuracy: 0.2734, test_Accuracy: 0.2684
Epoch: [ 0] [    4/  468] time: 3.3261, train_loss: 2.32175589, train_accuracy: 0.2812, test_Accuracy: 0.2899
Epoch: [ 0] [    5/  468] time: 3.7400, train_loss: 2.31768608, train_accuracy: 0.2656, test_Accuracy: 0.2860
Epoch: [ 0] [    6/  468] time: 4.0681, train_loss: 2.32552123, train_accuracy: 0.2031, test_Accuracy: 0.2640
Epoch: [ 0] [    7/  468] time: 4.4531, train_loss: 2.30419374, train_accuracy: 0.2969, test_Accuracy: 0.2530
Epoch: [ 0] [    8/  468] time: 4.8570,

Epoch: [ 0] [   74/  468] time: 30.3469, train_loss: 0.96117491, train_accuracy: 0.8672, test_Accuracy: 0.8162
Epoch: [ 0] [   75/  468] time: 30.7348, train_loss: 0.78676295, train_accuracy: 0.8984, test_Accuracy: 0.8164
Epoch: [ 0] [   76/  468] time: 31.1264, train_loss: 0.90882534, train_accuracy: 0.8203, test_Accuracy: 0.8138
Epoch: [ 0] [   77/  468] time: 31.4959, train_loss: 0.91708267, train_accuracy: 0.8438, test_Accuracy: 0.8115
Epoch: [ 0] [   78/  468] time: 31.8818, train_loss: 1.00982428, train_accuracy: 0.8359, test_Accuracy: 0.8079
Epoch: [ 0] [   79/  468] time: 32.2657, train_loss: 0.88015234, train_accuracy: 0.7812, test_Accuracy: 0.8058
Epoch: [ 0] [   80/  468] time: 32.6438, train_loss: 0.83739138, train_accuracy: 0.8047, test_Accuracy: 0.8034
Epoch: [ 0] [   81/  468] time: 33.0108, train_loss: 0.86377358, train_accuracy: 0.8359, test_Accuracy: 0.8004
Epoch: [ 0] [   82/  468] time: 33.4256, train_loss: 1.03534865, train_accuracy: 0.7656, test_Accuracy: 0.7985
E

Epoch: [ 0] [  148/  468] time: 58.0878, train_loss: 0.50775969, train_accuracy: 0.9453, test_Accuracy: 0.8991
Epoch: [ 0] [  149/  468] time: 58.4917, train_loss: 0.60249728, train_accuracy: 0.9141, test_Accuracy: 0.9012
Epoch: [ 0] [  150/  468] time: 58.8967, train_loss: 0.74238062, train_accuracy: 0.8750, test_Accuracy: 0.9023
Epoch: [ 0] [  151/  468] time: 59.2846, train_loss: 0.62166822, train_accuracy: 0.9141, test_Accuracy: 0.9038
Epoch: [ 0] [  152/  468] time: 59.7175, train_loss: 0.54413295, train_accuracy: 0.9297, test_Accuracy: 0.9047
Epoch: [ 0] [  153/  468] time: 60.0915, train_loss: 0.55255353, train_accuracy: 0.9141, test_Accuracy: 0.9047
Epoch: [ 0] [  154/  468] time: 60.4545, train_loss: 0.62872183, train_accuracy: 0.9141, test_Accuracy: 0.9054
Epoch: [ 0] [  155/  468] time: 60.7258, train_loss: 0.58441246, train_accuracy: 0.9297, test_Accuracy: 0.9051
Epoch: [ 0] [  156/  468] time: 61.0698, train_loss: 0.60400122, train_accuracy: 0.8828, test_Accuracy: 0.9054
E

Epoch: [ 0] [  222/  468] time: 84.9382, train_loss: 0.43380305, train_accuracy: 0.9375, test_Accuracy: 0.9220
Epoch: [ 0] [  223/  468] time: 85.3130, train_loss: 0.35789052, train_accuracy: 0.9609, test_Accuracy: 0.9232
Epoch: [ 0] [  224/  468] time: 85.7299, train_loss: 0.55081606, train_accuracy: 0.9297, test_Accuracy: 0.9237
Epoch: [ 0] [  225/  468] time: 86.1109, train_loss: 0.46000138, train_accuracy: 0.9375, test_Accuracy: 0.9252
Epoch: [ 0] [  226/  468] time: 86.4750, train_loss: 0.39291656, train_accuracy: 0.9531, test_Accuracy: 0.9255
Epoch: [ 0] [  227/  468] time: 86.8635, train_loss: 0.48931280, train_accuracy: 0.9297, test_Accuracy: 0.9256
Epoch: [ 0] [  228/  468] time: 87.2179, train_loss: 0.38858816, train_accuracy: 0.9453, test_Accuracy: 0.9252
Epoch: [ 0] [  229/  468] time: 87.5929, train_loss: 0.46537030, train_accuracy: 0.9609, test_Accuracy: 0.9256
Epoch: [ 0] [  230/  468] time: 87.8697, train_loss: 0.35856193, train_accuracy: 0.9688, test_Accuracy: 0.9258
E

Epoch: [ 0] [  296/  468] time: 112.1523, train_loss: 0.34668112, train_accuracy: 0.9297, test_Accuracy: 0.9333
Epoch: [ 0] [  297/  468] time: 112.6081, train_loss: 0.58235419, train_accuracy: 0.9141, test_Accuracy: 0.9312
Epoch: [ 0] [  298/  468] time: 113.0110, train_loss: 0.41287094, train_accuracy: 0.9219, test_Accuracy: 0.9296
Epoch: [ 0] [  299/  468] time: 113.3970, train_loss: 0.38198233, train_accuracy: 0.9609, test_Accuracy: 0.9303
Epoch: [ 0] [  300/  468] time: 113.7600, train_loss: 0.47457176, train_accuracy: 0.9062, test_Accuracy: 0.9312
Epoch: [ 0] [  301/  468] time: 114.1332, train_loss: 0.61284852, train_accuracy: 0.8906, test_Accuracy: 0.9330
Epoch: [ 0] [  302/  468] time: 114.5160, train_loss: 0.39403918, train_accuracy: 0.9453, test_Accuracy: 0.9347
Epoch: [ 0] [  303/  468] time: 114.9189, train_loss: 0.45382807, train_accuracy: 0.9453, test_Accuracy: 0.9354
Epoch: [ 0] [  304/  468] time: 115.2400, train_loss: 0.31359822, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 139.3895, train_loss: 0.27603006, train_accuracy: 0.9453, test_Accuracy: 0.9447
Epoch: [ 0] [  371/  468] time: 139.6870, train_loss: 0.35252723, train_accuracy: 0.9375, test_Accuracy: 0.9439
Epoch: [ 0] [  372/  468] time: 139.9433, train_loss: 0.31559718, train_accuracy: 0.9531, test_Accuracy: 0.9438
Epoch: [ 0] [  373/  468] time: 140.1989, train_loss: 0.49030733, train_accuracy: 0.9297, test_Accuracy: 0.9423
Epoch: [ 0] [  374/  468] time: 140.4800, train_loss: 0.28143597, train_accuracy: 0.9375, test_Accuracy: 0.9413
Epoch: [ 0] [  375/  468] time: 140.7219, train_loss: 0.22203507, train_accuracy: 0.9688, test_Accuracy: 0.9404
Epoch: [ 0] [  376/  468] time: 140.9745, train_loss: 0.37623233, train_accuracy: 0.9609, test_Accuracy: 0.9393
Epoch: [ 0] [  377/  468] time: 141.2485, train_loss: 0.41073036, train_accuracy: 0.8984, test_Accuracy: 0.9395
Epoch: [ 0] [  378/  468] time: 141.5098, train_loss: 0.43511215, train_accuracy: 0.9375, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 162.2294, train_loss: 0.27923036, train_accuracy: 0.9609, test_Accuracy: 0.9450
Epoch: [ 0] [  445/  468] time: 162.5556, train_loss: 0.37983847, train_accuracy: 0.9141, test_Accuracy: 0.9462
Epoch: [ 0] [  446/  468] time: 162.9854, train_loss: 0.37054172, train_accuracy: 0.9297, test_Accuracy: 0.9468
Epoch: [ 0] [  447/  468] time: 163.3504, train_loss: 0.37940657, train_accuracy: 0.9219, test_Accuracy: 0.9478
Epoch: [ 0] [  448/  468] time: 163.7653, train_loss: 0.32555604, train_accuracy: 0.9766, test_Accuracy: 0.9482
Epoch: [ 0] [  449/  468] time: 164.1393, train_loss: 0.43640503, train_accuracy: 0.9297, test_Accuracy: 0.9475
Epoch: [ 0] [  450/  468] time: 164.5475, train_loss: 0.38773441, train_accuracy: 0.9609, test_Accuracy: 0.9472
Epoch: [ 0] [  451/  468] time: 164.9641, train_loss: 0.29134279, train_accuracy: 0.9609, test_Accuracy: 0.9475
Epoch: [ 0] [  452/  468] time: 165.3600, train_loss: 0.26012951, train_accuracy: 0.9453, test_Accuracy:

### Accuracy : 95.91%